# Traffic sign recognition using Tensorflow 

For training I am using German sign dataset which can be found here: http://benchmark.ini.rub.de/

In [3]:
import os
import csv
import skimage.data
import skimage.transform
import matplotlib
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

Setting path to train and test data.

In [4]:
ROOT_PATH = "./datasets"
TRAIN_DATA_DIR = os.path.join(ROOT_PATH, "training_images/Final_Training/Images/")
TEST_DATA_DIR = os.path.join(ROOT_PATH, "testing_dataset/Final_Test/Images/")
IMAGE_WIDTH = 32
IMAGE_LENGTH = 32

Loading train data.

In [5]:
def load_train_data(data_dir):
    directories = [d for d in os.listdir(data_dir)
                  if os.path.isdir(os.path.join(data_dir, d))]
    labels = []
    images = []
    for d in directories:
        path = data_dir + d + '/'
        csv_file = open(path + 'GT-' + d + '.csv')
        temp_labels, temp_images = get_data(csv_file, path)
        labels += temp_labels
        images += temp_images
        
    return labels, images

Loading test data.

In [6]:
def load_test_data(data_dir):
    csv_file = open(data_dir + 'GT-final_test.csv')
        
    return get_data(csv_file, data_dir)

In [7]:
def get_data(csv_file, path):
    images = []
    labels = []
    reader = csv.reader(csv_file, delimiter=';')
    reader.next()
    for row in reader:
        images.append(skimage.transform.resize(plt.imread(path + row[0]), (IMAGE_WIDTH, IMAGE_LENGTH)))
        labels.append(int(row[7]))
    csv_file.close()
    return labels, images

In [8]:
labels, images = load_train_data(TRAIN_DATA_DIR)
test_labels, test_images = load_test_data(TEST_DATA_DIR)

In [9]:
print("Total training images {0}".format(len(images)))
print("Total testing images {0}".format(len(test_images)))

Total training images 42890
Total testing images 12630


Creating bar chart for dataset Visualization.

In [ ]:
data = {x:labels.count(x) for x in labels}.values()

plt.figure(figsize=(20, 10))

y_pos = np.arange(len(data))

plt.bar(y_pos, data, align='center', alpha = 0.5)
plt.xticks(y_pos, y_pos)
plt.axis('tight')
plt.ylabel("Number of examples")
plt.title("Training dataset")
plt.show()

Resize and normalize images.

In [ ]:
def normalize(image):
    return (image - image.min()) / (image.max() - image.min())

Difference between normal image and a normalized one.

In [ ]:
plt.figure(figsize = (10, 10))
plt.subplot(1, 2, 1)
plt.axis('off')
plt.title("Normal image")  
plt.imshow(images[1800])
plt.subplot(1, 2, 2)
plt.axis('off')
plt.title("Normalized image")
plt.imshow(normalize(images[1800]))
plt.show()

In [10]:
with tf.device('/device:GPU:0'):
    images_ph = tf.placeholder(tf.float32, [None, IMAGE_WIDTH, IMAGE_LENGTH, 3])
    labels_ph = tf.placeholder(tf.int64, [None])

    images_flat = tf.contrib.layers.flatten(images_ph)

    logits = tf.contrib.layers.fully_connected(images_flat, 44, tf.nn.elu)

    predicted = tf.nn.softmax(logits)
    predicted_labels = tf.argmax(predicted, 1)

    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits = logits,
                                                                   labels = labels_ph)
    loss = tf.reduce_mean(cross_entropy)

    optimizer = tf.train.AdamOptimizer(learning_rate=0.001).minimize(loss)

    correct_prediction = tf.equal(predicted_labels, labels_ph)
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

    init = tf.initialize_all_variables()

Instructions for updating:
Use `tf.global_variables_initializer` instead.


In [13]:
saver = tf.train.Saver()
session = tf.Session()
session.run(init)

In [14]:
labels_arr = np.array(labels)
images_arr = np.array(images)

In [15]:
feed_dict_train = {images_ph: images_arr,
             labels_ph: labels_arr}
feed_dict_test = {images_ph: test_images,
             labels_ph: test_labels}

In [18]:
for i in range(2001):
    loss_value = session.run([optimizer, loss], feed_dict_train)
    if i % 10 == 0:
        print("Loss: ", loss_value)

score = session.run(accuracy, feed_dict_test)
print("Accuracy: {:.3f}".format(score))

('Loss: ', [None, 2.0058527])
('Loss: ', [None, 1.9541161])
('Loss: ', [None, 1.907316])
('Loss: ', [None, 1.8645781])
('Loss: ', [None, 1.8249115])
('Loss: ', [None, 1.78714])
('Loss: ', [None, 1.7520248])
('Loss: ', [None, 1.7198185])
('Loss: ', [None, 1.6897899])
('Loss: ', [None, 1.6616679])
('Loss: ', [None, 1.6352246])
('Loss: ', [None, 1.6101419])
('Loss: ', [None, 1.585796])
('Loss: ', [None, 1.5624427])
('Loss: ', [None, 1.5405869])
('Loss: ', [None, 1.5199044])
('Loss: ', [None, 1.5001645])
('Loss: ', [None, 1.4807529])
('Loss: ', [None, 1.4534292])
('Loss: ', [None, 1.4288048])
('Loss: ', [None, 1.4081073])
('Loss: ', [None, 1.3898587])
('Loss: ', [None, 1.3730344])
('Loss: ', [None, 1.3572102])
('Loss: ', [None, 1.3422335])
('Loss: ', [None, 1.327931])
('Loss: ', [None, 1.3142248])
('Loss: ', [None, 1.3010418])
('Loss: ', [None, 1.2883035])
('Loss: ', [None, 1.2759398])
('Loss: ', [None, 1.2639649])
('Loss: ', [None, 1.252404])
('Loss: ', [None, 1.2412611])
('Loss: ', [None

In [19]:
saver.save(session, './model/model.ckpt')

'./model/model.ckpt'

In [ ]:
session.close()